# Importing Packages and creating universally helpful things

In [1]:
import numpy as np
import pandas as pd
import re
import requests
from collections import Counter
import bs4 as bs
import unicodedata
mdict = {"January":"1", "February":"2", "March":"3", "April":"4", "May":"5", "June":"6", "July":"7",\
         "August":"8", "September":"9", "October":"10", "November":"11", "December":"12"}
import us
STATES_DICT = us.states.mapping('name', 'abbr')

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)
def stripper(begins):
    begins = [strip_accents(boo) for boo in begins]
    begins = [re.sub(r"%C3%BA","u", u) for u in begins]
    begins = [re.sub(r"%C3%A9","e", e) for e in begins]
    begins = [re.sub(r"%C3%AD","i", i) for i in begins]
    begins = [re.sub(r"%C3%A1","a", a) for a in begins]
    begins = [re.sub("%27", "'", nra) for nra in begins]
    #begins = [re.sub(" [A-Z]\. ", " ", nra) for nra in begins]
    return(begins)



# 112th Congress (from January 3, 2011, until January 3, 2013)

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/112th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link.get('href')) for link in soup.find_all('a')]
begins = stripper(begins)
ends = []
for string in begins:
    if re.match(r"/wiki", string):
        ends.append(re.sub(r"/wiki/","",string))
sens112 = ends[237:390]
reps112 = ends[400:1331]
####
####
####
sstate = []
schamber = []
sname = []
rstate = []
rchamber = []
rname = []
for sen in sens112:
    if re.match(r"List_of.*", sen):
        state = re.sub(r"^List_of_United_States_Senators_from_","",sen)
    else:
        name = re.sub(r"_"," ", sen)
        name = re.sub(r"\(.*\)","",name)
        sname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        sstate.append(state)
        schamber.append("Senate")
for dis in reps112:
    if re.match(r"List_of.*", dis):
        state = re.sub(r"^List_of_United_States_Representatives_from_","",dis)
    #elif re.match(r".*_congressional_district$", dis):
    #    house115.append(state)
    #    temp = re.sub(r"_congressional_district$","",dis)
    #    house115.append("DISTRICT: " + re.sub(r"^.*[^_]+_","", temp))
    elif not re.match(r".*_congressional_district$", dis):
        name = re.sub(r"_"," ",dis)
        name = re.sub(r"\(.*\)","",name)
        rname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        rstate.append(state)
        rchamber.append("House")
rstates112 = pd.concat([pd.Series(rname), pd.Series(rchamber), pd.Series(rstate)], axis = 1)
rstates112.columns = ["Name", "Chamber", "State"] 
sstates112 = pd.concat([pd.Series(sname), pd.Series(schamber), pd.Series(sstate)], axis = 1)
sstates112.columns = ["Name", "Chamber", "State"] 

In [3]:
rstates112.head()

,Name,Chamber,State
0,Jo Bonner,House,Alabama
1,Martha Roby,House,Alabama
2,Mike D. Rogers,House,Alabama
3,Robert Aderholt,House,Alabama
4,Mo Brooks,House,Alabama


In [4]:
sstates112.head()

,Name,Chamber,State
0,Jeff Sessions,Senate,Alabama
1,Richard Shelby,Senate,Alabama
2,Mark Begich,Senate,Alaska
3,Lisa Murkowski,Senate,Alaska
4,Jon Kyl,Senate,Arizona


# 113th Congress (from January 3, 2013, to January 3, 2015)

In [5]:
source = requests.get("https://en.wikipedia.org/wiki/113th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link.get('href')) for link in soup.find_all('a')]
begins = stripper(begins)
ends = []
for string in begins:
    if re.match(r"/wiki", string):
        ends.append(re.sub(r"/wiki/","",string))
sens113 = ends[236:391]
reps113 = ends[402:1331]
####
####
####
sstate = []
schamber = []
sname = []
rstate = []
rchamber = []
rname = []
for sen in sens113:
    if re.match(r"List_of.*", sen):
        state = re.sub(r"^List_of_United_States_Senators_from_","",sen)
    else:
        name = re.sub(r"_"," ", sen)
        name = re.sub(r"\(.*\)","",name)
        sname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        sstate.append(state)
        schamber.append("Senate")
for dis in reps113:
    if re.match(r"List_of.*", dis):
        state = re.sub(r"^List_of_United_States_Representatives_from_","",dis)
    #elif re.match(r".*_congressional_district$", dis):
    #    house115.append(state)
    #    temp = re.sub(r"_congressional_district$","",dis)
    #    house115.append("DISTRICT: " + re.sub(r"^.*[^_]+_","", temp))
    elif not re.match(r".*_congressional_district$", dis):
        name = re.sub(r"_"," ",dis)
        name = re.sub(r"\(.*\)","",name)
        rname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        rstate.append(state)
        rchamber.append("House")
rstates113 = pd.concat([pd.Series(rname), pd.Series(rchamber), pd.Series(rstate)], axis = 1)
rstates113.columns = ["Name", "Chamber", "State"] 
sstates113 = pd.concat([pd.Series(sname), pd.Series(schamber), pd.Series(sstate)], axis = 1)
sstates113.columns = ["Name", "Chamber", "State"] 

In [6]:
rstates113.head()

,Name,Chamber,State
0,Jo Bonner,House,Alabama
1,Bradley Byrne,House,Alabama
2,Martha Roby,House,Alabama
3,Mike D. Rogers,House,Alabama
4,Robert Aderholt,House,Alabama


In [7]:
sstates113.head()

,Name,Chamber,State
0,Jeff Sessions,Senate,Alabama
1,Richard Shelby,Senate,Alabama
2,Mark Begich,Senate,Alaska
3,Lisa Murkowski,Senate,Alaska
4,Jeff Flake,Senate,Arizona


# 114th Congress (from January 3, 2015, to January 3, 2017)

In [8]:
source = requests.get("https://en.wikipedia.org/wiki/114th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link.get('href')) for link in soup.find_all('a')]
begins = stripper(begins)
ends = []
for string in begins:
    if re.match(r"/wiki", string):
        ends.append(re.sub(r"/wiki/","",string))
sens114 = ends[229:379]
reps114 = ends[390:1318]
####
####
####
sstate = []
schamber = []
sname = []
rstate = []
rchamber = []
rname = []
for sen in sens114:
    if re.match(r"List_of.*", sen):
        state = re.sub(r"^List_of_United_States_Senators_from_","",sen)
    else:
        name = re.sub(r"_"," ", sen)
        name = re.sub(r"\(.*\)","",name)
        sname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        sstate.append(state)
        schamber.append("Senate")
for dis in reps114:
    if re.match(r"List_of.*", dis):
        state = re.sub(r"^List_of_United_States_Representatives_from_","",dis)
    #elif re.match(r".*_congressional_district$", dis):
    #    house114.append(state)
    #    temp = re.sub(r"_congressional_district$","",dis)
    #    house114.append("DISTRICT: " + re.sub(r"^.*[^_]+_","", temp))
    elif not re.match(r".*_congressional_district$", dis):
        name = re.sub(r"_"," ",dis)
        name = re.sub(r"\(.*\)","",name)
        rname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        rstate.append(state)
        rchamber.append("House")
rstates114 = pd.concat([pd.Series(rname), pd.Series(rchamber), pd.Series(rstate)], axis = 1)
rstates114.columns = ["Name", "Chamber", "State"] 
sstates114 = pd.concat([pd.Series(sname), pd.Series(schamber), pd.Series(sstate)], axis = 1)
sstates114.columns = ["Name", "Chamber", "State"] 

In [9]:
rstates114.head()

,Name,Chamber,State
0,Bradley Byrne,House,Alabama
1,Martha Roby,House,Alabama
2,Mike Rogers,House,Alabama
3,Robert Aderholt,House,Alabama
4,Mo Brooks,House,Alabama


In [10]:
sstates114.head()

,Name,Chamber,State
0,Jeff Sessions,Senate,Alabama
1,Richard Shelby,Senate,Alabama
2,Dan Sullivan,Senate,Alaska
3,Lisa Murkowski,Senate,Alaska
4,Jeff Flake,Senate,Arizona


# 115th Congress (from January 3, 2017, to January 3, 2019)

In [11]:
source = requests.get("https://en.wikipedia.org/wiki/115th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link.get('href')) for link in soup.find_all('a')]
begins = stripper(begins)
ends = []
for string in begins:
    if re.match(r"/wiki", string):
        ends.append(re.sub(r"/wiki/","",string))
sens115 = ends[209:364]
reps115 = ends[376:1306]
####
####
####
sstate = []
schamber = []
sname = []
rstate = []
rchamber = []
rname = []
for sen in sens115:
    if re.match(r"List_of.*", sen):
        state = re.sub(r"^List_of_United_States_Senators_from_","",sen)
    else:
        name = re.sub(r"_"," ", sen)
        name = re.sub(r"\(.*\)","",name)
        sname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        sstate.append(state)
        schamber.append("Senate")
for dis in reps115:
    if re.match(r"List_of.*", dis):
        state = re.sub(r"^List_of_United_States_Representatives_from_","",dis)
    #elif re.match(r".*_congressional_district$", dis):
    #    house115.append(state)
    #    temp = re.sub(r"_congressional_district$","",dis)
    #    house115.append("DISTRICT: " + re.sub(r"^.*[^_]+_","", temp))
    elif not re.match(r".*_congressional_district$", dis):
        name = re.sub(r"_"," ",dis)
        name = re.sub(r"\(.*\)","",name)
        rname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        rstate.append(state)
        rchamber.append("House")
rstates115 = pd.concat([pd.Series(rname), pd.Series(rchamber), pd.Series(rstate)], axis = 1)
rstates115.columns = ["Name", "Chamber", "State"] 
sstates115 = pd.concat([pd.Series(sname), pd.Series(schamber), pd.Series(sstate)], axis = 1)
sstates115.columns = ["Name", "Chamber", "State"] 

In [12]:
rstates115.head()

,Name,Chamber,State
0,Bradley Byrne,House,Alabama
1,Martha Roby,House,Alabama
2,Mike Rogers,House,Alabama
3,Robert Aderholt,House,Alabama
4,Mo Brooks,House,Alabama


In [13]:
sstates115.head()

,Name,Chamber,State
0,Jeff Sessions,Senate,Alabama
1,Luther Strange,Senate,Alabama
2,Doug Jones,Senate,Alabama
3,Richard Shelby,Senate,Alabama
4,Dan Sullivan,Senate,Alaska


# Better Code

In [14]:
# 112th Congress (from January 3, 2011, until January 3, 2013)

In [15]:
source = requests.get("https://en.wikipedia.org/wiki/112th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link) for link in soup.find_all('li')]
begins = stripper(begins)
#begins = [re.sub(r"\'","a",a) for a in begins]
sens112 = begins[221:324]
reps112 = begins[375:824]
syear = ["<BLANK>" for x in range(len(sens112))]
sday = ["<BLANK>" for x in range(len(sens112))]
smonth = ["<BLANK>" for x in range(len(sens112))]
sparty = ["<BLANK>" for x in range(len(sens112))]
sname = ["<BLANK>" for x in range(len(sens112))]
sstate = ["<BLANK>" for x in range(len(sens112))]
##########
ryear = ["<BLANK>" for x in range(len(reps112))]
rday = ["<BLANK>" for x in range(len(reps112))]
rmonth = ["<BLANK>" for x in range(len(reps112))]
rparty = ["<BLANK>" for x in range(len(reps112))]
rname = ["<BLANK>" for x in range(len(reps112))]
rstate = ["<BLANK>" for x in range(len(reps112))]
rdistrict = ["<BLANK>" for x in range(len(reps112))]
hmm = [re.sub(r"<[^>]+>","",sen) for sen in sens112]
for sen in range(len(hmm)):
    z = re.match("^3\..*",hmm[sen]) #class 3 begins this congress
    if z:
        syear[sen] = "2010" #election year
        smonth[sen] = "11" #election month
        sday[sen] = "2" #election day
        name = re.sub("^3\. ","",hmm[sen]) #Senate cycle according to wikipedia metadata
        sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(sname[sen],"",name)
        party = re.findall(r"\([^ ]+\)", party)[0]
        sparty[sen] = re.sub(r"[^A-Z]+","",party)
for sen in range(len(hmm)):
    if syear[sen] == "<BLANK>":
        if re.match("^[^0-9].*",hmm[sen]):
            step2 = re.findall(r"[A-Za-z]+ [0-9]+, \d\d\d\d",hmm[sen])[0]
            sday[sen] = re.sub(r",","",re.findall(r"[0-9]+,",step2)[0])
            syear[sen] = re.findall(r"\d\d\d\d", step2)[0]
            smonth[sen] = mdict[re.sub(r"[0-9, ]+","", step2)]
            name = re.sub(r"\[[^ ]+\]","",hmm[sen])
            sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
            party = re.sub(sname[sen],"",name)
            sparty[sen] = re.sub(r"[^A-Z]+","",party)
syear = [x for x in syear if x!= "<BLANK>"]
smonth = [x for x in smonth if x!= "<BLANK>"]
sday = [x for x in sday if x!= "<BLANK>"]
sname = [x for x in sname if x!= "<BLANK>"]
sparty = [x[0] for x in sparty if x!= "<BLANK>"]
schamber = ["Senate" for x in sparty if x!="<BLANK>"]
df = pd.DataFrame({'year': syear,
                       'month': smonth,
                       'day': sday})
sdate = pd.to_datetime(df)
senate112 = pd.concat([sdate, pd.Series(sparty), pd.Series(sname), pd.Series(schamber)], axis = 1)
senate112.columns = ["Date", "Party", "Name", "Chamber"]
#######
#######
#######
hmm = [re.sub(r"<[^>]+>","",rep) for rep in reps112]
hmm = [re.sub("[Aa]t\-large\.","0.",rep) for rep in hmm]
for rep in range(len(hmm)):
    z = re.match("^[0-9]+\..*",hmm[rep])
    if z:
        district = re.findall(r"^[0-9]+", hmm[rep])[0]
        rdistrict[rep] = district
        name = re.sub(rdistrict[rep] + ". ","", hmm[rep])
        rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(rname[rep] + " ","",name)
        party = re.findall(r"^\([^ ]+\)",party)[0]
        rparty[rep] = re.sub(r"[\(\)]","", party)
        ryear[rep] = "2010" #election year
        rmonth[rep] = "11" #election month
        rday[rep] = "2" #election day   
    else:
        if not re.search(".*Vacant.*", hmm[rep]):
            rdistrict[rep] = district
            rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", hmm[rep])[0]
            party = re.sub(rname[rep] + " ","", hmm[rep])
            party = re.findall(r"^\([^ ]+\)",party)[0]
            rparty[rep] = re.sub(r"[\(\)]","", party)
            date = re.sub("^.*from ","",hmm[rep])
            ryear[rep] = re.findall(r"\d\d\d\d",date)[0]
            date = re.sub(ryear[rep],"",date)
            date = re.sub(r",.*$","",date)
            rday[rep] = re.findall(r"[0-9]+",date)[0]
            date = re.sub(" " + rday[rep],"",date)
            rmonth[rep] = mdict[date]
ryear = [x for x in ryear if x!= "<BLANK>"]
rmonth = [x for x in rmonth if x!= "<BLANK>"]
rday = [x for x in rday if x!= "<BLANK>"]
rname = [x for x in rname if x!= "<BLANK>"]
rparty = [x[0] for x in rparty if x!= "<BLANK>"]
rchamber = ["House" for x in rparty if x!= "<BLANK>"]
df = pd.DataFrame({'year': ryear,
                       'month': rmonth,
                       'day': rday})
rdate = pd.to_datetime(df)
house112 = pd.concat([rdate, pd.Series(rparty), pd.Series(rname), pd.Series(rchamber), pd.Series(rdistrict)], axis = 1)
house112.columns = ["Date", "Party", "Name", "Chamber", "District"]  

In [16]:
senate112.head()

,Date,Party,Name,Chamber
0,2010-11-02,R,Richard Shelby,Senate
1,2010-11-02,R,Lisa Murkowski,Senate
2,2010-11-02,R,John McCain,Senate
3,2010-11-02,R,John Boozman,Senate
4,2010-11-02,D,Barbara Boxer,Senate


In [17]:
house112.head()

,Date,Party,Name,Chamber,District
0,2010-11-02,R,Jo Bonner,House,1
1,2010-11-02,R,Martha Roby,House,2
2,2010-11-02,R,Mike Rogers,House,3
3,2010-11-02,R,Robert Aderholt,House,4
4,2010-11-02,R,Mo Brooks,House,5


In [18]:
# 113th Congress (from January 3, 2013, to January 3, 2015)

In [19]:
source = requests.get("https://en.wikipedia.org/wiki/113th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link) for link in soup.find_all('li')]
begins = stripper(begins)
sens113 = begins[237:342]
reps113 = begins[393:837]
syear = ["<BLANK>" for x in range(len(sens113))]
sday = ["<BLANK>" for x in range(len(sens113))]
smonth = ["<BLANK>" for x in range(len(sens113))]
sparty = ["<BLANK>" for x in range(len(sens113))]
sname = ["<BLANK>" for x in range(len(sens113))]
sstate = ["<BLANK>" for x in range(len(sens113))]
##########
ryear = ["<BLANK>" for x in range(len(reps113))]
rday = ["<BLANK>" for x in range(len(reps113))]
rmonth = ["<BLANK>" for x in range(len(reps113))]
rparty = ["<BLANK>" for x in range(len(reps113))]
rname = ["<BLANK>" for x in range(len(reps113))]
rstate = ["<BLANK>" for x in range(len(reps113))]
rdistrict = ["<BLANK>" for x in range(len(reps113))]
hmm = [re.sub(r"<[^>]+>","",sen) for sen in sens113]
for sen in range(len(hmm)):
    z = re.match("^1\..*",hmm[sen]) #class 2 begins this congress
    if z:
        syear[sen] = "2012" #election year
        smonth[sen] = "11" #election month
        sday[sen] = "6" #election day
        name = re.sub("^1\. ","",hmm[sen]) #Senate cycle according to wikipedia metadata
        sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(sname[sen],"",name)
        party = re.findall(r"\([^ ]+\)", party)[0]
        sparty[sen] = re.sub(r"[^A-Z]+","",party)
for sen in range(len(hmm)):
    if syear[sen] == "<BLANK>":
        if re.match("^[^0-9].*",hmm[sen]):
            step2 = re.findall(r"[A-Za-z]+ [0-9]+, \d\d\d\d",hmm[sen])[0]
            sday[sen] = re.sub(r",","",re.findall(r"[0-9]+,",step2)[0])
            syear[sen] = re.findall(r"\d\d\d\d", step2)[0]
            smonth[sen] = mdict[re.sub(r"[0-9, ]+","", step2)]
            name = re.sub(r"\[[^ ]+\]","",hmm[sen])
            sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
            party = re.sub(sname[sen],"",name)
            sparty[sen] = re.sub(r"[^A-Z]+","",party)
syear = [x for x in syear if x!= "<BLANK>"]
smonth = [x for x in smonth if x!= "<BLANK>"]
sday = [x for x in sday if x!= "<BLANK>"]
sname = [x for x in sname if x!= "<BLANK>"]
sparty = [x[0] for x in sparty if x!= "<BLANK>"]
schamber = ["Senate" for x in sparty if x!= "<BLANK>"]
df = pd.DataFrame({'year': syear,
                       'month': smonth,
                       'day': sday})
sdate = pd.to_datetime(df)
senate113 = pd.concat([sdate, pd.Series(sparty), pd.Series(sname), pd.Series(schamber)], axis = 1)
senate113.columns = ["Date", "Party", "Name", "Chamber"]
#######
#######
#######
hmm = [re.sub(r"<[^>]+>","",rep) for rep in reps113]
hmm = [re.sub("[Aa]t\-large\.","0.",rep) for rep in hmm]
for rep in range(len(hmm)):
    z = re.match("^[0-9]+\..*",hmm[rep])
    if z:
        district = re.findall(r"^[0-9]+", hmm[rep])[0]
        rdistrict[rep] = district
        name = re.sub(rdistrict[rep] + ". ","", hmm[rep])
        rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(rname[rep] + " ","",name)
        party = re.findall(r"^\([^ ]+\)",party)[0]
        rparty[rep] = re.sub(r"[\(\)]","", party)
        ryear[rep] = "2012" #election year
        rmonth[rep] = "11" #election month
        rday[rep] = "6" #election day   
    else:
        if not re.search(".*Vacant.*", hmm[rep]):
            rdistrict[rep] = district
            rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", hmm[rep])[0]
            party = re.sub(rname[rep] + " ","", hmm[rep])
            party = re.findall(r"^\([^ ]+\)",party)[0]
            rparty[rep] = re.sub(r"[\(\)]","", party)
            date = re.sub("^.*from ","",hmm[rep])
            ryear[rep] = re.findall(r"\d\d\d\d",date)[0]
            date = re.sub(ryear[rep],"",date)
            date = re.sub(r",.*$","",date)
            rday[rep] = re.findall(r"[0-9]+",date)[0]
            date = re.sub(" " + rday[rep],"",date)
            rmonth[rep] = mdict[date]
ryear = [x for x in ryear if x!= "<BLANK>"]
rmonth = [x for x in rmonth if x!= "<BLANK>"]
rday = [x for x in rday if x!= "<BLANK>"]
rname = [x for x in rname if x!= "<BLANK>"]
rparty = [x[0] for x in rparty if x!= "<BLANK>"]
rchamber = ["House" for x in rparty if x!= "<BLANK>"]
df = pd.DataFrame({'year': ryear,
                       'month': rmonth,
                       'day': rday})
rdate = pd.to_datetime(df)
house113 = pd.concat([rdate, pd.Series(rparty), pd.Series(rname), pd.Series(rchamber), pd.Series(rdistrict)], axis = 1)
house113.columns = ["Date", "Party", "Name", "Chamber", "District"]  

In [20]:
senate113.head()

,Date,Party,Name,Chamber
0,2012-11-06,R,Jeff Flake,Senate
1,2012-11-06,D,Dianne Feinstein,Senate
2,2012-11-06,D,Chris Murphy,Senate
3,2012-11-06,D,Tom Carper,Senate
4,2012-11-06,D,Bill Nelson,Senate


In [21]:
house113.head()

,Date,Party,Name,Chamber,District
0,2012-11-06,R,Jo Bonner,House,1
1,2014-01-07,R,Bradley Byrne,House,1
2,2012-11-06,R,Martha Roby,House,2
3,2012-11-06,R,Mike Rogers,House,3
4,2012-11-06,R,Robert Aderholt,House,4


In [22]:
# 114th Congress (from January 3, 2015, to January 3, 2017)

In [23]:
source = requests.get("https://en.wikipedia.org/wiki/114th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link) for link in soup.find_all('li')]
begins = stripper(begins)
sens114 = begins[223:323]
reps114 = begins[374:818]
syear = ["<BLANK>" for x in range(len(sens114))]
sday = ["<BLANK>" for x in range(len(sens114))]
smonth = ["<BLANK>" for x in range(len(sens114))]
sparty = ["<BLANK>" for x in range(len(sens114))]
sname = ["<BLANK>" for x in range(len(sens114))]
sstate = ["<BLANK>" for x in range(len(sens114))]
##########
ryear = ["<BLANK>" for x in range(len(reps114))]
rday = ["<BLANK>" for x in range(len(reps114))]
rmonth = ["<BLANK>" for x in range(len(reps114))]
rparty = ["<BLANK>" for x in range(len(reps114))]
rname = ["<BLANK>" for x in range(len(reps114))]
rstate = ["<BLANK>" for x in range(len(reps114))]
rdistrict = ["<BLANK>" for x in range(len(reps114))]
hmm = [re.sub(r"<[^>]+>","",sen) for sen in sens114]
for sen in range(len(hmm)):
    z = re.match("^2\..*",hmm[sen]) #class 2 begins this congress
    if z:
        syear[sen] = "2014" #election year
        smonth[sen] = "11" #election month
        sday[sen] = "4" #election day
        name = re.sub("^2\. ","",hmm[sen]) #Senate cycle according to wikipedia metadata
        sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(sname[sen],"",name)
        party = re.findall(r"\([^ ]+\)", party)[0]
        sparty[sen] = re.sub(r"[^A-Z]+","",party)
for sen in range(len(hmm)):
    if syear[sen] == "<BLANK>":
        if re.match("^[^0-9].*",hmm[sen]):
            step2 = re.findall(r"[A-Za-z]+ [0-9]+, \d\d\d\d",hmm[sen])[0]
            sday[sen] = re.sub(r",","",re.findall(r"[0-9]+,",step2)[0])
            syear[sen] = re.findall(r"\d\d\d\d", step2)[0]
            smonth[sen] = mdict[re.sub(r"[0-9, ]+","", step2)]
            name = re.sub(r"\[[^ ]+\]","",hmm[sen])
            sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
            party = re.sub(sname[sen],"",name)
            sparty[sen] = re.sub(r"[^A-Z]+","",party)
syear = [x for x in syear if x!= "<BLANK>"]
smonth = [x for x in smonth if x!= "<BLANK>"]
sday = [x for x in sday if x!= "<BLANK>"]
sname = [x for x in sname if x!= "<BLANK>"]
sparty = [x[0] for x in sparty if x!= "<BLANK>"]
schamber = ["Senate" for x in sparty if x!= "<BLANK>"]
df = pd.DataFrame({'year': syear,
                       'month': smonth,
                       'day': sday})
sdate = pd.to_datetime(df)
senate114 = pd.concat([sdate, pd.Series(sparty), pd.Series(sname), pd.Series(schamber)], axis = 1)
senate114.columns = ["Date", "Party", "Name", "Chamber"]
#######
#######
#######
hmm = [re.sub(r"<[^>]+>","",rep) for rep in reps114]
hmm = [re.sub("[Aa]t\-large\.","0.",rep) for rep in hmm]
for rep in range(len(hmm)):
    z = re.match("^[0-9]+\..*",hmm[rep])
    if z:
        district = re.findall(r"^[0-9]+", hmm[rep])[0]
        rdistrict[rep] = district
        name = re.sub(rdistrict[rep] + ". ","", hmm[rep])
        rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(rname[rep] + " ","",name)
        party = re.findall(r"^\([^ ]+\)",party)[0]
        rparty[rep] = re.sub(r"[\(\)]","", party)
        ryear[rep] = "2014" #election year
        rmonth[rep] = "11" #election month
        rday[rep] = "4" #election day   
    else:
        if not re.search(".*Vacant.*", hmm[rep]):
            rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", hmm[rep])[0]
            rdistrict[rep] = district
            party = re.sub(rname[rep] + " ","", hmm[rep])
            party = re.findall(r"^\([^ ]+\)",party)[0]
            rparty[rep] = re.sub(r"[\(\)]","", party)
            date = re.sub("^.*from ","",hmm[rep])
            ryear[rep] = re.findall(r"\d\d\d\d",date)[0]
            date = re.sub(ryear[rep],"",date)
            date = re.sub(r",.*$","",date)
            rday[rep] = re.findall(r"[0-9]+",date)[0]
            date = re.sub(" " + rday[rep],"",date)
            rmonth[rep] = mdict[date]
ryear = [x for x in ryear if x!= "<BLANK>"]
rmonth = [x for x in rmonth if x!= "<BLANK>"]
rday = [x for x in rday if x!= "<BLANK>"]
rname = [x for x in rname if x!= "<BLANK>"]
rparty = [x[0] for x in rparty if x!= "<BLANK>"]
rchamber = ["House" for x in rparty if x!="<BLANK>"]
rdistrict = [x for x in rdistrict if x!="<BLANK>"]
df = pd.DataFrame({'year': ryear,
                       'month': rmonth,
                       'day': rday})
rdate = pd.to_datetime(df)
house114 = pd.concat([rdate, pd.Series(rparty), pd.Series(rname), pd.Series(rchamber), pd.Series(rdistrict)], axis = 1)
house114.columns = ["Date", "Party", "Name", "Chamber", "District"]   

In [24]:
senate114.head()

,Date,Party,Name,Chamber
0,2014-11-04,R,Jeff Sessions,Senate
1,2014-11-04,R,Dan Sullivan,Senate
2,2014-11-04,R,Tom Cotton,Senate
3,2014-11-04,R,Cory Gardner,Senate
4,2014-11-04,D,Chris Coons,Senate


In [25]:
house114.head()

,Date,Party,Name,Chamber,District
0,2014-11-04,R,Bradley Byrne,House,1
1,2014-11-04,R,Martha Roby,House,2
2,2014-11-04,R,Mike Rogers,House,3
3,2014-11-04,R,Robert Aderholt,House,4
4,2014-11-04,R,Mo Brooks,House,5


In [26]:
# 115th Congress (from January 3, 2017, to January 3, 2019)

In [27]:
source = requests.get("https://en.wikipedia.org/wiki/115th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link) for link in soup.find_all('li')]
begins = stripper(begins)
sens115 = begins[256:361]
reps115 = begins[412:864]
syear = ["<BLANK>" for x in range(len(sens115))]
sday = ["<BLANK>" for x in range(len(sens115))]
smonth = ["<BLANK>" for x in range(len(sens115))]
sparty = ["<BLANK>" for x in range(len(sens115))]
sname = ["<BLANK>" for x in range(len(sens115))]
sstate = ["<BLANK>" for x in range(len(sens115))]
##########
ryear = ["<BLANK>" for x in range(len(reps115))]
rday = ["<BLANK>" for x in range(len(reps115))]
rmonth = ["<BLANK>" for x in range(len(reps115))]
rparty = ["<BLANK>" for x in range(len(reps115))]
rname = ["<BLANK>" for x in range(len(reps115))]
rstate = ["<BLANK>" for x in range(len(reps115))]
rdistrict = ["<BLANK>" for x in range(len(reps115))]
hmm = [re.sub(r"<[^>]+>","",sen) for sen in sens115]
for sen in range(len(hmm)):
    z = re.match("^3\..*",hmm[sen]) #class 3 begins this congress
    if z:
        syear[sen] = "2016" #election year
        smonth[sen] = "11" #election month
        sday[sen] = "8" #election day
        name = re.sub("^3\. ","",hmm[sen]) #Senate cycle according to wikipedia metadata
        sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(sname[sen],"",name)
        party = re.findall(r"\([^ ]+\)", party)[0]
        sparty[sen] = re.sub(r"[^A-Z]+","",party)
for sen in range(len(hmm)):
    if syear[sen] == "<BLANK>":
        if re.match("^[^0-9].*",hmm[sen]):
            step2 = re.findall(r"[A-Za-z]+ [0-9]+, \d\d\d\d",hmm[sen])[0]
            sday[sen] = re.sub(r",","",re.findall(r"[0-9]+,",step2)[0])
            syear[sen] = re.findall(r"\d\d\d\d", step2)[0]
            smonth[sen] = mdict[re.sub(r"[0-9, ]+","", step2)]
            name = re.sub(r"\[[^ ]+\]","",hmm[sen])
            sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
            party = re.sub(sname[sen],"",name)
            sparty[sen] = re.sub(r"[^A-Z]+","",party)
syear = [x for x in syear if x!= "<BLANK>"]
smonth = [x for x in smonth if x!= "<BLANK>"]
sday = [x for x in sday if x!= "<BLANK>"]
sname = [x for x in sname if x!= "<BLANK>"]
sparty = [x[0] for x in sparty if x!= "<BLANK>"]
schamber = ["Senate" for x in sparty if x!= "<BLANK>"]
df = pd.DataFrame({'year': syear,
                       'month': smonth,
                       'day': sday})
sdate = pd.to_datetime(df)
senate115 = pd.concat([sdate, pd.Series(sparty), pd.Series(sname), pd.Series(schamber)], axis = 1)
senate115.columns = ["Date", "Party", "Name", "Chamber"]
#######
#######
#######
hmm = [re.sub(r"<[^>]+>","",rep) for rep in reps115]
hmm = [re.sub("[Aa]t\-large\.","0.",rep) for rep in hmm]
for rep in range(len(hmm)):
    z = re.match("^[0-9]+\..*",hmm[rep])
    if z:
        district = re.findall(r"^[0-9]+", hmm[rep])[0]
        rdistrict[rep] = district
        name = re.sub(rdistrict[rep] + ". ","", hmm[rep])
        rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(rname[rep] + " ","",name)
        party = re.findall(r"^\([^ ]+\)",party)[0]
        rparty[rep] = re.sub(r"[\(\)]","", party)
        ryear[rep] = "2016" #election year
        rmonth[rep] = "11" #election month
        rday[rep] = "8" #election day   
    else:
        if not re.search(".*Vacant.*", hmm[rep]):
            rdistrict[rep] = district
            rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", hmm[rep])[0]
            party = re.sub(rname[rep] + " ","", hmm[rep])
            party = re.findall(r"^\([^ ]+\)",party)[0]
            rparty[rep] = re.sub(r"[\(\)]","", party)
            date = re.sub("^.*from ","",hmm[rep])
            ryear[rep] = re.findall(r"\d\d\d\d",date)[0]
            date = re.sub(ryear[rep],"",date)
            date = re.sub(r",.*$","",date)
            rday[rep] = re.findall(r"[0-9]+",date)[0]
            date = re.sub(" " + rday[rep],"",date)
            rmonth[rep] = mdict[date]
            rdistrict[rep] = district
ryear = [x for x in ryear if x!= "<BLANK>"]
rmonth = [x for x in rmonth if x!= "<BLANK>"]
rday = [x for x in rday if x!= "<BLANK>"]
rname = [x for x in rname if x!= "<BLANK>"]
rparty = [x[0] for x in rparty if x!= "<BLANK>"]
rchamber = ["House" for x in rparty if x!= "<BLANK>"]
rdistrict = [x for x in rdistrict if x!="<BLANK>"]
df = pd.DataFrame({'year': ryear,
                       'month': rmonth,
                       'day': rday})
rdate = pd.to_datetime(df)
house115 = pd.concat([rdate, pd.Series(rparty), pd.Series(rname), pd.Series(rchamber), pd.Series(rdistrict)], axis = 1)
house115.columns = ["Date", "Party", "Name", "Chamber", "District"]


In [28]:
senate115.head()

,Date,Party,Name,Chamber
0,2017-02-09,R,Luther Strange,Senate
1,2018-01-03,D,Doug Jones,Senate
2,2016-11-08,R,Richard Shelby,Senate
3,2016-11-08,R,Lisa Murkowski,Senate
4,2016-11-08,R,John McCain,Senate


In [29]:
house115.head()

,Date,Party,Name,Chamber,District
0,2016-11-08,R,Bradley Byrne,House,1
1,2016-11-08,R,Martha Roby,House,2
2,2016-11-08,R,Mike Rogers,House,3
3,2016-11-08,R,Robert Aderholt,House,4
4,2016-11-08,R,Mo Brooks,House,5


# Mighty Joins !

In [30]:
def checker(x,y):
    okay  = pd.merge(x, y,  how='outer', left_on=['Name','Chamber'], right_on = ['Name','Chamber'])
    weird = okay[okay.isnull().any(axis=1)]
    return(len(weird))
def wurry(x,y):
    okay  = pd.merge(x, y,  how='outer', left_on=['Name','Chamber'], right_on = ['Name','Chamber'])
    weird = okay[okay.isnull().any(axis=1)]
    return(weird)
def okee(x,y):
    okay  = pd.merge(x, y,  how='inner', left_on=['Name','Chamber'], right_on = ['Name','Chamber'])
    return(len(okay))
def hurry(x,y):
    return(pd.merge(x, y,  how='inner', left_on=['Name','Chamber'], right_on = ['Name','Chamber']))


In [31]:
senate = pd.concat([hurry(senate115,sstates115),\
                     hurry(senate114,sstates114),\
                     hurry(senate113,sstates113),\
                     hurry(senate112,sstates112)], axis = 0)
senate = senate.reset_index(drop = True)
senate["State"] = senate["State"].str.replace("_"," ")
senate["State"] = senate["State"].map(STATES_DICT)
senate["Party"] = senate["Party"].str.lower()
senate["Name"] = senate["Name"].str.lower()
senate["Name"] = senate["Name"].str.replace(r"[^a-z ]","")
senate["Chamber"] = senate["Chamber"].str.lower()
senate.columns = ["date", "party", "name", "chamber", "state"]
######
######
######
house = pd.concat([hurry(house115,rstates115),\
                     hurry(house114,rstates114),\
                     hurry(house113,rstates113),\
                     hurry(house112,rstates112)], axis = 0)
house = house.reset_index(drop = True)
house["State"] = house["State"].str.replace("_"," ")
house["State"] = house["State"].map(STATES_DICT)
house["Party"] = house["Party"].str.lower()
house["Name"] = house["Name"].str.lower()
house["Name"] = house["Name"].str.replace(r"[^a-z ]","")
house["Chamber"] = house["Chamber"].str.lower()
house.columns = ["date", "party", "name", "chamber", "district", "state"]
house = house[["date", "party", "name", "chamber","state", "district"]]

In [32]:
house

,date,party,name,chamber,state,district
0,2016-11-08,r,bradley byrne,house,AL,1
1,2016-11-08,r,martha roby,house,AL,2
2,2016-11-08,r,mike rogers,house,AL,3
3,2016-11-08,r,robert aderholt,house,AL,4
4,2016-11-08,r,mo brooks,house,AL,5
5,2016-11-08,r,gary palmer,house,AL,6
6,2016-11-08,d,terri sewell,house,AL,7
7,2016-11-08,r,don young,house,AK,0
8,2016-11-08,d,tom o'halleran,house,AZ,1
9,2016-11-08,r,martha mcsally,house,AZ,2


In [33]:
house.tail()

,date,party,name,chamber,state,district
1721,2010-11-02,r,jim sensenbrenner,house,WI,1
1722,2010-11-02,r,tom petri,house,WI,2
1723,2010-11-02,r,sean duffy,house,WI,3
1724,2010-11-02,r,reid ribble,house,WI,4
1725,2010-11-02,r,cynthia lummis,house,WY,5


In [34]:
len(house)

1726

In [35]:
senate

,date,party,name,chamber,state
0,2017-02-09,r,luther strange,senate,AL
1,2018-01-03,d,doug jones,senate,AL
2,2016-11-08,r,richard shelby,senate,AL
3,2016-11-08,r,lisa murkowski,senate,AK
4,2016-11-08,r,john mccain,senate,AZ
5,2018-09-04,r,jon kyl,senate,AZ
6,2016-11-08,r,john boozman,senate,AR
7,2016-11-08,d,kamala harris,senate,CA
8,2016-11-08,d,michael bennet,senate,CO
9,2016-11-08,d,richard blumenthal,senate,CT


In [36]:
len(senate)

147

In [37]:
#house.to_csv("Clean_House_Winners.csv")
#senate.to_csv("Clean_Senate_Winners.csv")